<a href="https://colab.research.google.com/github/kdpbiiitd/CSE508_Winter2023_A2_39./blob/main/IR_A2_Improvements.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import string
import nltk
import math
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from collections import defaultdict
import numpy as np

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [4]:
df = pd.read_csv('dataset.csv')
#df = df.drop('ArticleId', axis=1)

print(df.head())

   ArticleId                                               Text  Category
0       1833  worldcom ex-boss launches defence lawyers defe...  business
1        154  german business confidence slides german busin...  business
2       1101  bbc poll indicates economic gloom citizens in ...  business
3       1976  lifestyle  governs mobile choice  faster  bett...      tech
4        917  enron bosses in $168m payout eighteen former e...  business


In [5]:
stopwords = nltk.corpus.stopwords.words('english')
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

def clean_text(text):
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = " ".join([word.lower() for word in text.split() if word.lower() not in stopwords])
    return text

In [6]:
df['Text'] = df['Text'].apply(clean_text)
df['Text'] = df['Text'].apply(word_tokenize)
df['Text'] = df['Text'].apply(lambda x: [stemmer.stem(word) for word in x])
df['Text'] = df['Text'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])

In [7]:
# Calculate TF
tf = defaultdict(Counter)
for i, row in df.iterrows():
    for word in row['Text']:
        tf[row['Category']][word] += 1

# Calculate ICF
N = len(df)
icf = {}
for word in set(df['Text'].sum()):
    n_t = sum([1 for i, row in df.iterrows() if word in row['Text']])
    icf[word] = math.log(N / n_t)

# Calculate TF-ICF
tf_icf = defaultdict(dict)
for label, counter in tf.items():
    for word, count in counter.items():
        tf_icf[label][word] = count * icf[word]

In [8]:
# Create a list of dictionaries
X = df['Text'].apply(lambda x: dict(Counter(x))).tolist()

In [ ]:
print(df['Category'].unique())

['business' 'tech' 'politics' 'sport' 'entertainment']


In [31]:
# Split the dataset into training and testing sets using a 70:30 split
X_train, X_test, y_train, y_test = train_test_split(X, df['Category'], test_size=0.3, random_state=42)

# Create a MultinomialNB classifier
clf = MultinomialNB()

# Create a DictVectorizer object
vec = DictVectorizer()

# Fit the vectorizer on the training set
vec.fit(X_train)

# Transform the training set
X_train = vec.transform(X_train)

# Transform the testing set
X_test = vec.transform(X_test)

# Fit the classifier on the training set
clf.fit(X_train, y_train)

MultinomialNB()

In [32]:
# Make predictions on the testing data
y_pred = clf.predict(X_test)

# Calculate performance metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

# Print results
print(f'Accuracy: {accuracy:.2f}')
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1 score: {f1:.2f}')

Accuracy: 0.98
Precision: 0.98
Recall: 0.98
F1 score: 0.98
